# Publication database
Create a database that tracks publications and their authors in order

In [ ]:
import datajoint as dj

In [ ]:
schema = dj.schema('dimitri_publications')
schema.drop()
schema = dj.schema('dimitri_publications')

In [ ]:
@schema
class Publication(dj.Manual):
    definition = """
    pub_id : int 
    ---
    title : varchar(1000)
    publish_date  : date 
    author : varchar(1000)
    """

In [ ]:
import faker
faker = faker.Faker()

In [ ]:
for i in range(30):
    Publication.insert1(dict(
        pub_id=i, title=faker.sentence(), 
        publish_date=faker.date_this_decade(), author=faker.name()), skip_duplicates=True)

In [ ]:
Publication()

# New design. Many-many publication/author

In [ ]:
schema.drop()
schema = dj.schema('dimitri_publications')

In [ ]:
@schema 
class Author(dj.Manual):
    definition = """
    author_id : int 
    ---
    first_name : varchar(100)
    last_name : varchar(100)
    """

In [ ]:
for i in range(100):
    Author.insert1(dict(author_id=i, 
                        first_name=faker.first_name(), 
                        last_name=faker.last_name()))

### Do the same kind of insert in using SQL

In [ ]:
# Insert into the Author table using pymysql and SQL statements
import json
with open('cred.json') as f:
    creds=json.load(f)
    
import pymysql

# establish a database connection
conn = pymysql.connect(
    host=creds['host'], 
    user=creds['user'], 
    passwd=creds['password'],
    autocommit=True)

cursor = conn.cursor()
cursor.execute("""SELECT * FROM dimitri_publications.author""")

In [ ]:
cursor.fetchall()

In [ ]:
cursor.execute("""
INSERT INTO dimitri_publications.author (author_id, first_name, last_name) VALUES (%s, %s, %s)
""", (1001, faker.first_name(), faker.last_name()))

In [ ]:
# same thing but faster
Author.insert((
    dict(
        author_id=i, 
        first_name=faker.first_name(), 
        last_name=faker.last_name()) for i in range(100))
    , skip_duplicates=True)

In [ ]:
Author()

In [ ]:
@schema
class Publication(dj.Manual):
    definition = """
    pub_id : int 
    ---
    title : varchar(1000)
    publish_date  : date 
    """

In [ ]:
Publication.insert([dict(
    pub_id=i, title=faker.sentence(), 
    publish_date=faker.date_this_decade()) for i in range(30)], skip_duplicates=True)

In [ ]:
Publication()

In [ ]:
@schema
class PubAuthor(dj.Manual):
    definition = """
    -> Publication
    -> Author
    """

In [ ]:
authors = Author.fetch("KEY")
pubs = Publication.fetch("KEY")

In [ ]:
import random

In [ ]:
for pub in pubs:
    for auth in random.sample(authors, random.randint(1, 3)):
        PubAuthor.insert1({**pub, **auth}, skip_duplicates=True)

In [ ]:
PubAuthor()

In [ ]:
Publication * PubAuthor * Author & 'pub_id=6'

In [ ]:
dj.Diagram(schema)

# Another design: Track the order of authors on the publication

In [ ]:
PubAuthor.drop()

In [ ]:
@schema
class PubAuthor(dj.Manual):
    definition = """
    -> Publication
    -> Author
    ---
    author_order : tinyint 
    unique index(pub_id, author_order)
    """

In [ ]:
for pub in pubs:
    for author_order, auth in enumerate(random.choices(authors, k=random.randint(1, 3))):
        PubAuthor.insert1(dict(**pub, **auth, author_order=author_order+1), skip_duplicates=True)

In [ ]:
PubAuthor()

# Cities, States, and Capitals

Design a database with
- Cities
- States
- State capitals


### Rules
1. A city belongs to a state
2. A state capital is a city in the same state
3. Only one city can be capital in each state
4. Cities and states have permanents names
5. Cities have unique names in each state

In [ ]:
schema = dj.schema('dimitri_states')
schema.drop()  # drop to clear history
schema = dj.schema('dimitri_states')

In [ ]:
@schema
class State(dj.Manual):
    definition = """
    state : char(2)  # TX, CA, etc
    ---
    state_name : varchar(25)
    """
    
State.insert((
    ("CA", "California"),
    ("TX", "Texas"),
    ("NY", "New York"),
    ("DE", "Delaware"),
    ("NE", "Nebraska"),
    ("MI", "Michigan"),
    ("WI", "Wisconsin"),
    ("PA", "Pensylvania")
), skip_duplicates=True)

In [ ]:
State()

In [ ]:
@schema
class City(dj.Manual):
    definition = """
    -> State  
    city_id : int
    ---
    city_name : varchar(100)  
    unique index (state, city_name)
    """

@schema
class Capital(dj.Manual):
    definition = """
    -> State
    ---
    -> City    
    """

#### Equivalent SQL to declare City

```sql
CREATE TABLE dimitri_state.city (
  state char(2) NOT NULL,
  city_id int NOT NULL,
  city_name varchar(100) NOT NULL,
  PRIMARY KEY (state, city_id),
  UNIQUE INDEX (state, city_name),
  FOREIGN KEY (state) REFERENCES state(state)
)
```

#### Equivalent SQL to declare Capital

```sql
CREATE TABLE dimitri_state.capital (
  state char(2) NOT NULL,
  city_id int NOT NULL,
  PRIMARY KEY (state),
  FOREIGN KEY (state, city_id) REFERENCE city(state, city_id),
  FOREIGN KEY (state) REFERENCES state(state)
)
```

In [ ]:
City.insert(
    (
        ("TX", 1, "Houston"),
        ("TX", 2, "Austin"),
        ("DE", 1, "Dover"),
        ("DE", 2, "Wilmington"),
        ("CA", 1, "Sacramento"),
        ("CA", 2, "Houston"),
        ("CA", 3, "Los Angeles"),
        ("WI", 1, "Milwaukee"),
        ("WI", 2, "Madison"),
        ("CA", 4, "Madison")
    ))

Capital.insert((
    ("WI", 1),
    ("CA", 1),
    ("DE", 2),
    ("TX", 2),
))

In [ ]:
Capital()

In [ ]:
dj.Diagram(schema)

In [ ]:
Capital()

# Bank accounts and transfers

Design a database with 

- Customers
- Accounts with a current balance
- Deposits / Withdrawals 
- Transfers between accounts 
- Account balnace history

### Rules

1. A customer can have any number of accounts.
1. An account can have many customers
1. Overdrafts are not allowed: when attempting a withdrawal or transfer, the current balance is not allowed to go negative.
1. For a transfer, the credit from one account must match the debit from the other.

In [ ]:
import faker
import random
fake = faker.Faker()

In [ ]:
schema = dj.schema('dimitri_bank')
schema.drop()
schema = dj.schema('dimitri_bank')

In [ ]:
@schema
class Customer(dj.Manual):
    definition = """
    customer_id : int
    ---
    customer_name : varchar(100)
    ssn  :  char(11) 
    unique index (ssn) 
    """

In [ ]:
Customer.insert((
    {'customer_id': i, 'customer_name': fake.name(), 'ssn': fake.ssn()}
               for i in range(30)))

In [ ]:
Customer()

In [ ]:
@schema
class Account(dj.Manual):
    definition = """
    account_id : int
    ---
    account_type : enum("savings", "checking")
    open_date : date    
    """
    
@schema
class AccountCustomer(dj.Manual):
    definition = """
    -> Customer
    -> Account
    """    

In [ ]:
dj.Diagram(schema)

In [ ]:
Account.insert(
    ({'account_id': i, 
      'open_date': fake.date_this_decade(), 
      'account_type': random.choice(("checking", "savings"))} for i in range(20)))

In [ ]:
customers = Customer.fetch('KEY')
for a in Account.fetch('KEY'):
    AccountCustomer.insert(
        {**a, **c} for c in random.sample(customers, random.randint(1, 3)))

In [ ]:
@schema
class Balance(dj.Manual):
    definition = """
    -> Account 
    ---
    amount : decimal(9, 2)
    """

In [ ]:
dj.Diagram(schema)

In [ ]:
for a in Account.fetch('KEY'):
    Balance.insert1(dict(a, amount=random.randint(0, 120000)/100))

In [ ]:
Balance()

In [ ]:
def withdraw(key, amount):
    conn = dj.conn()
    with conn.transaction:
        previous_amount = (Balance & key).fetch1('amount')
        if amount > previous_amount: 
            raise ValueError('Insufficient funds')
        (Balance & key).delete_quick()
        # an error or disruption can happen here
        Balance.insert1(dict(key, amount=float(previous_amount) - amount))
    print('Remaining balance: {balance}'.format(balance=(Balance & key).fetch1('amount')))

In [ ]:
key = {'account_id': 3}
withdraw(key, 50.00)

# Homework:  Hotel Database -- Due Nov 21.

Design a database with 

- Rooms
- Guests 
- Reservations
- Check-in:  date/time  for reservation
- Check-out: date/time  for reservation

### Rules
1. A reservation reserves one room for one night for one guest
2. A guest can reserve multiple rooms per night.
2. A guest must have a reservation to check in.
3. Checkout can only be done after a check-in. 
4. Checking into a room is allowed only after the previous guest checks out 

### Assignment
1. Design tables that enforce these rules
2. Populate with fake rooms, customers, and nights.
3. Define function `reserve` to make a reservation. Call it to make fake reservations.
3. Define functions `checkin` and `checkout`. Call them to demonstrate that they work. 
4. Use transactions if or when necessary